In [5]:
import pandas as pd

## Functions

#### Merging

In [126]:
MERGE_COLUMNS = ["year", "SA2 code"]

def get_merged_df(new_df, final_df):
    # get the attributes used for the merge
    merge_columns = list(set(new_df.columns) & set(MERGE_COLUMNS))

    # merge columns
    merged_df = pd.merge(final_df, new_df, on=merge_columns, how="inner")

    # check if any columns lost
    if (merged_df.shape[0] < final_df.shape[0]):
        print(f"lost {final_df.shape[0] - merged_df.shape[0]} out of {final_df.shape[0]} records")
    
    return merged_df

### General

In [107]:
def rename_dict(df, rename_dict, keep_columns):
    # get the renamed dictionary
    new_column_names = list(df.columns.copy())
    for index, new_name in rename_dict.items():
        new_column_names[index - 1] = new_name
    df.columns = new_column_names

    # drop the columns not of interest
    keep_columns = [x-1 for x in keep_columns]
    df = df.iloc[:, keep_columns]

    return df

### Preparing suburb name

In [46]:
import numpy as np

DIRECTIONS = ["north", "south", "west", "east"]

"""`df` should have unique indicies and"""
def suburb_name_split(df):
    indicies = list(df.index)
    final_indicies = []
    repeated_indicies = []
    counter = 0

    print(len(indicies))

    # for each index, gather how many indicies to repeat and the final product
    for index in indicies:
        # remove (.vic) if present
        clean_index = index.replace("(Vic.)", "")

        # replace the brackets around "west"
        clean_index = clean_index.replace("(", "").replace(")", "")

        # make sure lower case, no confusion
        clean_index = clean_index.lower()

        # nothing fancy
        if (not "-" in clean_index):
            final_indicies.append(clean_index)
            repeated_indicies.append(index)
            continue
        
        counter += 1

        # need to split
        index_split = clean_index.split(" - ")

        if (len(index_split) != 2):
            print("length of index split is not 2?")
            exit()
        
        # if only one direction, reintroduce without the weird thing
        if (index_split[1].lower() in DIRECTIONS):
            final_indicies.append(" ".join(index_split))
            repeated_indicies.append(index)
        
        # else will need to strip everything
        else:
            final_indicies.extend(index_split)
            repeated_indicies.extend(np.repeat(index, 2))
    
    print(counter / len(indicies))

    # duplicate
    df = df.loc[repeated_indicies]

    print(df.shape)
    print(len(final_indicies))

    # change the names accordingly
    df.index = final_indicies
    
    return df


### Column and imputation

In [6]:
def impute_previous(columns, na):
    flat_column_list = [[group[i] for group in columns] for i in range(len(columns[0]))]

    column_dataframe = pd.DataFrame(flat_column_list)

    column_dataframe = column_dataframe.applymap(lambda x: None if (not x) or (na in str(x)) else x)

    column_dataframe = column_dataframe.T.ffill().T

    flat_column_list = column_dataframe.to_numpy()

    return flat_column_list

In [7]:
def fill_columns(df, column_dict, na="Unnamed"):
    columns = list(df.columns)

    for index, new_column in column_dict.items():
        columns[index] = new_column

    if (type(df.columns) == pd.MultiIndex):
        flat_column_list = impute_previous(columns, na)

        df.columns = pd.MultiIndex.from_arrays(flat_column_list)
    
    else:
        df.columns = columns

    return df

## Economic by Region (and beginning)

In [8]:
def filter_victoria(df, column_name):
    region_mask = ~df[column_name].str.isnumeric()

    # get the region indexes and names
    regions_index, regions_names = list(df[region_mask].index), list(df[region_mask][column_name].values)

    # find the index for victoria in the list
    index_victoria = regions_names.index("Victoria")

    # reference the mask between victoria and the next index
    df = df.loc[regions_index[index_victoria] + 1: regions_index[index_victoria + 1] - 1]

    return df

### Geography summary

In [427]:
COLUMNS_DICT_INCOME_GEOGRAPHY = {
    0: ("SA2 code",None),
    1: ("SA2 name",None)
}

In [428]:
# get the df
final_df = pd.read_csv("./data/landing/economic_by_region/income_by_geography_b2022.csv", header=[0, 1])

# fill empty columns for the join
final_df = fill_columns(final_df, COLUMNS_DICT_INCOME_GEOGRAPHY)

# only look at columns of interest
final_df = filter_victoria(final_df, ("SA2 code", None))

# create a massive list of stuff, each record has a unique year, location and measure
final_df = final_df.melt(id_vars=final_df.columns.to_list()[:2], 
                         value_vars = final_df.columns.to_list()[2:],
                         var_name=["Measure", "Year"],
                         value_name="Value")

# make it so each record only has a unique year and location
final_df = final_df.pivot(index=list(final_df.columns[:2]) + ["Year"], columns="Measure", values="Value")
final_df = final_df.reset_index()
final_df.columns = [x[0] for x in final_df.columns[:2]] + list(final_df.columns[2:])

In [429]:
# filter and rename
NEW_NAMES = {
    3: "year",
    4: "economic: number of earners",
    6: "economic: median income", 
    7: "economic: median age of earners"
}
KEEP_COLUMNS = [1, 2, 3, 4, 6, 7]

final_df = rename_dict(final_df, NEW_NAMES, KEEP_COLUMNS)
print(final_df.shape[0])
final_df.head(5)

2610


,SA2 code,SA2 name,year,economic: number of earners,economic: median income,economic: median age of earners
0,201011001,Alfredton,2016-17,"7,117","50,596",42
1,201011001,Alfredton,2017-18,"7,558","52,448",42
2,201011001,Alfredton,2018-19,"7,987","53,932",42
3,201011001,Alfredton,2019-20,"8,665","55,204",41
4,201011001,Alfredton,2020-21,"9,438","58,036",41


In [430]:
import numpy as np

MEASURE_COLUMNS = final_df.columns[3:]

# drop any na values in measure columns
final_df = final_df.replace("np", np.nan)
final_df = final_df.dropna(subset=MEASURE_COLUMNS, how="any")
final_df.shape[0]

# convert to first year
final_df["year"] = final_df["year"].apply(lambda x: x[:4]).astype(int)

# get numeric columns
for measure_column in MEASURE_COLUMNS:
    final_df[measure_column] = pd.to_numeric(final_df[measure_column].str.replace(',', ''))

print(final_df.shape[0])
final_df.head(5)

2602


,SA2 code,SA2 name,year,economic: number of earners,economic: median income,economic: median age of earners
0,201011001,Alfredton,2016,7117,50596,42
1,201011001,Alfredton,2017,7558,52448,42
2,201011001,Alfredton,2018,7987,53932,42
3,201011001,Alfredton,2019,8665,55204,41
4,201011001,Alfredton,2020,9438,58036,41


In [431]:
# REMOVE SA2 regions without all the years

# get the regions with 5 counts
SA2_year_counts = final_df.groupby(["SA2 code"])["year"].count()
SA2_regions = SA2_year_counts[SA2_year_counts == 5].index

# remove anything doesn't have these from the final_df
final_df = final_df[final_df["SA2 code"].isin(SA2_regions)]
final_df.shape[0]

2595

In [432]:
START_YEAR = 2016
END_YEAR = 2021

for curr_year in range(START_YEAR, END_YEAR):
    # get the years
    curr_year_df = final_df[(final_df["year"] == curr_year)]
    both_years_df = final_df[(final_df["year"] == curr_year) | (final_df["year"] == curr_year + 1)]

    # average using the next
    avg_year_df = both_years_df[["SA2 code"] + list(MEASURE_COLUMNS)].groupby("SA2 code").mean()

    # only concerned about SA2 regions of current year
    avg_year_df = avg_year_df.loc[curr_year_df["SA2 code"].unique()]

    # get the indices of the original year
    avg_year_df.index = curr_year_df.index

    # assign the new chunk
    final_df.loc[curr_year_df.index, MEASURE_COLUMNS] = avg_year_df

final_df.head(5)


,SA2 code,SA2 name,year,economic: number of earners,economic: median income,economic: median age of earners
0,201011001,Alfredton,2016,7337.5,51522.0,42.0
1,201011001,Alfredton,2017,7772.5,53190.0,42.0
2,201011001,Alfredton,2018,8326.0,54568.0,41.5
3,201011001,Alfredton,2019,9051.5,56620.0,41.0
4,201011001,Alfredton,2020,9438.0,58036.0,41.0


### Distribution

In [433]:
# get the economic distribtuion
economic_distribution_df = pd.read_csv("./data/landing/economic_by_region/income_distribution_by_geography_2021.csv", header=0)
economic_distribution_df = economic_distribution_df.drop(0)
economic_distribution_df.head(5)

# rename and filter
NEW_NAMES = {
    1: "SA2 code", 
    13: "economic: top 1% suburb",
    14: "economic: top 5% suburb"
}
KEEP_COLUMNS = [1, 12, 13, 14]

economic_distribution_df = rename_dict(economic_distribution_df, NEW_NAMES, KEEP_COLUMNS)
economic_distribution_df.head()

,SA2 code,Gini coefficient,economic: top 1% suburb,economic: top 5% suburb
1,Australia,0.483,9.8,22.5
2,New South Wales,0.497,11.2,24.3
3,101021007,0.615,21.2,32.6
4,101021008,0.365,4.4,14.5
5,101021009,0.368,4.6,14.7


In [434]:
final_df = get_merged_df(economic_distribution_df, final_df)
final_df.head(5)

,SA2 code,SA2 name,year,economic: number of earners,economic: median income,economic: median age of earners,Gini coefficient,economic: top 1% suburb,economic: top 5% suburb
0,201011001,Alfredton,2016,7337.5,51522.0,42.0,0.438,6.4,18.2
1,201011001,Alfredton,2017,7772.5,53190.0,42.0,0.438,6.4,18.2
2,201011001,Alfredton,2018,8326.0,54568.0,41.5,0.438,6.4,18.2
3,201011001,Alfredton,2019,9051.5,56620.0,41.0,0.438,6.4,18.2
4,201011001,Alfredton,2020,9438.0,58036.0,41.0,0.438,6.4,18.2


## Economic

In [149]:
final_df.columns

Index(['SA2 code', 'SA2 name', 'year', 'economic: number of earners',
       'economic: median income', 'economic: median age of earners',
       'Gini coefficient', 'economic: top 1% suburb',
       'economic: top 5% suburb'],
      dtype='object')

## Housing